# Custom BERT Keras Layer

A simple example to demonstrate how to create a Keras model which includes BERT as a layer.

In [1]:
# GPU selection
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras import layers

import utils
import bert_utils

In [3]:
BERT_PATH = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
#BERT_PATH = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"

MAX_SEQ_LEN = 512

In [4]:
config = tf.ConfigProto()
# allow us to instrument the GPU VRAM usage
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

## Creating the Model

### Model Inputs

In [5]:
in_id = layers.Input(shape=(MAX_SEQ_LEN,), name="input_ids")
in_mask = layers.Input(shape=(MAX_SEQ_LEN,), name="input_masks")
in_segment = layers.Input(shape=(MAX_SEQ_LEN,), name="segment_ids")

in_bert = [in_id, in_mask, in_segment]

### BERT Layer

In [9]:
l_bert = bert_utils.BERT(fine_tune_layers=0,
                         bert_path=BERT_PATH,
                         return_sequence=True,
                         output_size=768,
                         debug=True)(in_bert)

print("Output", l_bert.shape)

['layer_0', 'layer_1', 'layer_10', 'layer_11', 'layer_2', 'layer_3', 'layer_4', 'layer_5', 'layer_6', 'layer_7', 'layer_8', 'layer_9']
BERT module loaded with 12 Transformer cells, training all cells > 12
DEBUG: Printing trainable vars
Name                                                 | Shape         | Params
Output (?, ?, 768)


In [15]:
x = layers.Reshape((MAX_SEQ_LEN, 768))(l_bert)
x = layers.Conv1D(512, 1)(x)
x = layers.Flatten()(x)
x.shape

TensorShape([Dimension(None), Dimension(262144)])

In [16]:
x = layers.Dense(1, activation="relu", kernel_initializer="he_uniform")(x)
x.shape

TensorShape([Dimension(None), Dimension(1)])

In [ ]:
model = tf.keras.models.Model(inputs=in_bert, outputs=out_pred)

In [ ]:
model.summary()